## 分布式训练

随着模型参数和训练数据的急速增长，单机已经无法满足模型的训练需求，因此需要设计分布式训练系统来解决海量的计算和内存资源要求问题。本节将会依次介绍机器学习系统的基础概念、分布式训练集群架构、分布式训练并行策略，并以DeepSpeed为例介绍如何在大集群上训练大语言模型。

### 分布式训练概述

`分布式训练`是指将机器学习或深度学习模型训练任务分解成多个子任务，并在多个计算设备上并行地进行训练。计算设备可以是中央处理器（Central Processing Unit，CPU）、图形处理器（Graphics Processing Unit，GPU）、张量处理器（Tensor Processing Unit，TPU）也可以是神经网络处理器（Neural network Processing Unit，NPU）。  

<img src="./images/multi_device.png" style="zoom:70%;" /> 

由于同一个服务器内部的多个计算设备(单机多卡)之间内存也可能并不共享，因此无论这些计算设备是否处于同一个服务器还是多个服务器中，其系统架构都属于分布式系统范畴。  

单设备计算速度主要由单块计算加速芯片的`运算速度`和`数据I/O能力`来决定，对单设备训练效率进行优化，主要的技术手段有混合精度训练、算子融合、梯度累加等；分布式训练系统中计算设备数量越多，其理论峰值计算速度就会越高，但是受到通讯效率的影响，计算设备数量增大则会造成加速比急速降低；多设备加速比则是由计算和`通讯效率`决定，需要结合算法和网络拓扑结构进行优化，分布式训练并行策略主要目标就是提升分布式训练系统中的多设备加速比。  



### 分布式训练并行策略

单节点模型训练系统主要由数据和模型两部分组成，训练过程会有多个数据小批次(mini-batch)完成。训练系统利用批次数据通过多个算子(神经网络层)根据损失函数和优化算法来生成梯度，从而调整模型参数。整个模型训练的过程是可以有一个计算图来表示:  

<img src="./images/single_train_system.png" style="zoom:70%;" /> 

计算图的执行过程可以分为前向计算和反向计算两个阶段。前向计算就是将批次数据通过计算图中的每个算子，然后反向计算是根据损失函数和优化算法，每个算子计算梯度，并更新本地的参数。这样根据批次一轮一轮的进行前向传播和反向传播计算数据、更新参数。

根据上面所述，很容易想到如果进行并行加速，可以从模型和数据两个维度来考虑。1.可以对数据进行切分，同时将模型复制到多个设备上，并行执行不同的数据分片，这种方式成为数据并行；2.可以对模型进行切分，将模型中的算子分发到多个设备分别完成，称为模型并行；3.可以结合上面说的两种并行方式，称为混合并行。

#### 数据并行

在数据并行系统中，每个计算设备都有整个模型的副本，在训练时每个计算设备都会分配一个批次数据样本的子集，进行前向传播计算。在前向计算完成后，每个计算设备需要计算所有算子的梯度，并将本地梯度进行广播。需要聚合设备的本地梯度后然后取均值对模型进行更新，完成该批次的训练。  

<img src="./images/data_parallel.png" style="zoom:70%;" /> 

数据并行训练系统可以通过增加计算设备，有效提升整体训练吞吐量，即`每秒全局批次数`。与单计算设备的区别在于反向传播阶段的梯度需要在所有设备完成同步，并计算平均梯度来更新参数。TensorFlow和Pytorch都有对应的实现方法，TensorFlow DistributedStrategy、PyTorch Distributed、Horovod DistributedOptimizer。

数据并行训练加速比最高，同时显存占用也比较高，主要是要求每个设备上都备份一分模型导致的。

In [ ]:
# 使用 PyTorch DistributedDataParallel 实现单个服务器多加速卡训练代码

# from torch.utils.data.distributed import DistributedSampler

# DistributedSampler类

import math
from typing import TypeVar, Optional, Iterator

import torch
from . import Sampler, Dataset
import torch.distributed as dist

__all__ = ["DistributedSampler", ]

T_co = TypeVar('T_co', covariant=True)


class DistributedSampler(Sampler[T_co]):
    r"""Sampler that restricts data loading to a subset of the dataset.

    It is especially useful in conjunction with
    :class:`torch.nn.parallel.DistributedDataParallel`. In such a case, each
    process can pass a :class:`~torch.utils.data.DistributedSampler` instance as a
    :class:`~torch.utils.data.DataLoader` sampler, and load a subset of the
    original dataset that is exclusive to it.

    .. note::
        Dataset is assumed to be of constant size and that any instance of it always
        returns the same elements in the same order.

    Args:
        dataset: Dataset used for sampling.
        num_replicas (int, optional): Number of processes participating in
            distributed training. By default, :attr:`world_size` is retrieved from the
            current distributed group.
        rank (int, optional): Rank of the current process within :attr:`num_replicas`.
            By default, :attr:`rank` is retrieved from the current distributed
            group.
        shuffle (bool, optional): If ``True`` (default), sampler will shuffle the
            indices.
        seed (int, optional): random seed used to shuffle the sampler if
            :attr:`shuffle=True`. This number should be identical across all
            processes in the distributed group. Default: ``0``.
        drop_last (bool, optional): if ``True``, then the sampler will drop the
            tail of the data to make it evenly divisible across the number of
            replicas. If ``False``, the sampler will add extra indices to make
            the data evenly divisible across the replicas. Default: ``False``.

    .. warning::
        In distributed mode, calling the :meth:`set_epoch` method at
        the beginning of each epoch **before** creating the :class:`DataLoader` iterator
        is necessary to make shuffling work properly across multiple epochs. Otherwise,
        the same ordering will be always used.

    Example::

        >>> # xdoctest: +SKIP
        >>> sampler = DistributedSampler(dataset) if is_distributed else None
        >>> loader = DataLoader(dataset, shuffle=(sampler is None),
        ...                     sampler=sampler)
        >>> for epoch in range(start_epoch, n_epochs):
        ...     if is_distributed:
        ...         sampler.set_epoch(epoch)
        ...     train(loader)
    """

    def __init__(self, dataset: Dataset, num_replicas: Optional[int] = None,
                 rank: Optional[int] = None, shuffle: bool = True,
                 seed: int = 0, drop_last: bool = False) -> None:
        if num_replicas is None:
            if not dist.is_available():
                raise RuntimeError("Requires distributed package to be available")
            num_replicas = dist.get_world_size()
        if rank is None:
            if not dist.is_available():
                raise RuntimeError("Requires distributed package to be available")
            rank = dist.get_rank()
        if rank >= num_replicas or rank < 0:
            raise ValueError(
                "Invalid rank {}, rank should be in the interval"
                " [0, {}]".format(rank, num_replicas - 1))
        self.dataset = dataset
        self.num_replicas = num_replicas
        self.rank = rank
        self.epoch = 0
        self.drop_last = drop_last
        # If the dataset length is evenly divisible by # of replicas, then there
        # is no need to drop any data, since the dataset will be split equally.
        if self.drop_last and len(self.dataset) % self.num_replicas != 0:  # type: ignore[arg-type]
            # Split to nearest available length that is evenly divisible.
            # This is to ensure each rank receives the same amount of data when
            # using this Sampler.
            self.num_samples = math.ceil(
                (len(self.dataset) - self.num_replicas) / self.num_replicas  # type: ignore[arg-type]
            )
        else:
            self.num_samples = math.ceil(len(self.dataset) / self.num_replicas)  # type: ignore[arg-type]
        self.total_size = self.num_samples * self.num_replicas
        self.shuffle = shuffle
        self.seed = seed

    def __iter__(self) -> Iterator[T_co]:
        if self.shuffle:
            # deterministically shuffle based on epoch and seed
            g = torch.Generator()
            g.manual_seed(self.seed + self.epoch)
            indices = torch.randperm(len(self.dataset), generator=g).tolist()  # type: ignore[arg-type]
        else:
            indices = list(range(len(self.dataset)))  # type: ignore[arg-type]

        if not self.drop_last:
            # add extra samples to make it evenly divisible
            padding_size = self.total_size - len(indices)
            if padding_size <= len(indices):
                indices += indices[:padding_size]
            else:
                indices += (indices * math.ceil(padding_size / len(indices)))[:padding_size]
        else:
            # remove tail of data to make it evenly divisible.
            indices = indices[:self.total_size]
        assert len(indices) == self.total_size

        # subsample
        indices = indices[self.rank:self.total_size:self.num_replicas]
        assert len(indices) == self.num_samples

        return iter(indices)

    def __len__(self) -> int:
        return self.num_samples

    def set_epoch(self, epoch: int) -> None:
        r"""
        Sets the epoch for this sampler. When :attr:`shuffle=True`, this ensures all replicas
        use a different random ordering for each epoch. Otherwise, the next iteration of this
        sampler will yield the same ordering.

        Args:
            epoch (int): Epoch number.
        """
        self.epoch = epoch

In [ ]:
# 具体使用的一个demo见下面的git代码 - train_multi_gpu.py文件
# https://github.com/TianyuJIAA/llm-course/tree/main/%E5%B0%9A%E7%A1%85%E8%B0%B7/chapter01/BertClassifier

#### 模型并行

模型并行主要用于解决单点内存不足的问题。比如GPT-3的模型参数是1750亿，模型参数都通过32位浮点数表示，则模型需要占用700G内存，计算公式为:  
```text
1 GB = 10^3 MB = 10^6 KB = 10^9 Bytes
32bit = 4Bytes
1750*10^8*4 = 175*4%10^9 = 700GB
```
H100加速也仅支持80GB显存，所以无法将模型完整放入其中。模型并行从计算图角度可以从两种方式切分:  
1.按模型的层切分到不同设备，即层并行或算子间并行，也称之为流水线并行  
2.将计算图层内的参数切分到不同设备，即层内并行或算子间并行，也称为张量并行。  

<img src="./images/model_parallel.png" style="zoom:70%;" /> 

##### 流水线并行

`流水线`并行是一种并行计算策略，将模型的各个层分段处理，并将每个段分布到不同的计算设备上，使得前后阶段如流水线一样工作。  

典型的流水线并行系统如图所示，包含了前向和后向计算，这种并行系统中的设备长时间处于空闲状态，使用率大幅降低，形成了模型并行气泡，也称为流水线气泡。

<img src="./images/pipeline_bubble.png" style="zoom:70%;" /> 

针对朴素流水线并行系统使用率低的问题，提出GPipe方法。将小批次进一步划分为更小的微批次，利用流水线并行方案，每次处理一个微批次的数据。通过这种方式有效地降低了并行气泡，但是只有当一个Mini-batch中所有的前向计算完成才能够开始执行后向计算，也降低了系统的并行效率。  

<img src="./images/Gpipe.png" style="zoom:70%;" /> 

1F1B流水线策略引入了任务调度机制，使下游设备在等待上游计算的同时执行其他可并行的任务，提高设备的利用率。  

<img src="./images/1F1B.png" style="zoom:70%;" /> 

Pytorch中包含了实现流水线的API函数Pipe，具体参考: torch.distributed.pipeline.sync.Pipe  

```python
# Step 0. Need to initialize RPC framework first.
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '29500'
torch.distributed.rpc.init_rpc('worker', rank=0, world_size=1)

# Step 1: build a model including two linear layers
fc1 = nn.Linear(16, 8).cuda(0)
fc2 = nn.Linear(8, 4).cuda(1)

# Step 2: wrap the two layers with nn.Sequential
model = nn.Sequential(fc1, fc2)

# Step 3: build Pipe (torch.distributed.pipeline.sync.Pipe)
model = Pipe(model, chunks=8)

# do training/inference
input = torch.rand(16, 16).cuda(0)
output_rref = model(input) 
```

##### 张量并行

`张量并行`需要根据模型的具体结构和算子类型，解决如何将参数切分到不同设备，以及如何保证切分后数学一致性问题。其实本质上就是将矩阵按行或者按列进行切分到不同的设备中，然后分别进行矩阵的相乘，最后通过通信得到完整的全量结果。  

具体以Transformer结构为基础，其主要由以下三种算子组成: Embedding、矩阵乘和交叉熵损失计算构成。  

1.对于嵌入式表示算子，可以将词表进行按照词维度进行切分，每个设备只存储部分词向量，然后通过汇总各个设备的部分词向量，得到完整的词向量  

<img src="./images/tensor_embedding.png" style="zoom:70%;" /> 

2.矩阵乘的张量并行要充分利用矩阵分块乘法原理，也就是按照行或者是按照列进行切分。

<img src="./images/tensor_multi.png" style="zoom:70%;" /> 

将矩阵`A`按照行或者列切分为A1和A2，并放到不同的设备上，然后分别将矩阵X与其相乘，最后多个计算设备之间进行通信，并拼接在一起得到最终的结果矩阵Y  

3.分类层会通过softmax和交叉熵来计算损失。如果类别数量特别大，也会导致设备内存无法存储和计算logi矩阵。所以可以按照类别维度切分，同时通过中间结果通信，得到最终的全局的交叉熵损失

最后附上多头注意力机制张量并行示意图:  

<img src="./images/tensor_mlh.png" style="zoom:70%;" /> 

In [ ]:
# Pytorch提供了细粒度张量级别的并行API，DistributedTensor，也提供了粗粒度模型层面的API可以对"nn.Module"进行张量并行。
# 这个from torch.distributed._tensor引用有问题先跳过
import torch
from torch.distributed._tensor import DTensor, DeviceMesh, Shard, distribute_tensor # type: ignore

# construct a device mesh with available devices (multi-host or single host)
device_mesh = DeviceMesh("cuda", [0, 1, 2, 3])

# if we want to do row-wise sharding
rowwise_placement=[Shard(0)]

# if we want to do col-wise sharding
colwise_placement=[Shard(1)]
big_tensor = torch.randn(888, 12)

# distributed tensor returned will be sharded across the dimension specified in placements
rowwise_tensor = distribute_tensor(big_tensor, device_mesh=device_mesh, placements=rowwise_placement)


##### 混合并行

混合并行是将多种并行策略如数据并行、流水线并行和张量并行等进行混合使用。通过结合不同的并行策略，混合并行可以发挥各种并行策略的优点，最大程度地提高计算性能和效率。具体来说:  
1.通常在每个`服务器内部`使用`张量并行`策略，由于该策略涉及的网络通信量较大，需要利用服务器内部的不同计算设备之间进行高速通信带宽。  
2.通过`流水线并行`，将模型的不同层划分为多个阶段，每个阶段由`不同的机器`负责计算。这样可以充分利用多台机器的计算能力，并通过机器之间的高速通信来传递计算结果和中间数据，以提高整体的计算速度和效率。  
3.最后，在外层叠加数据并行策略，以增加并发数量，提升整体训练速度。通过`数据并行`，将训练数据分发到多组服务器上进行并行处理， `每组服务器`处理不同的数据批次。这样可以充分利用多台服务器的计算资源，并增加训练的并发度，从而加快整体训练速度。  

`BLOOM`是支持最多59种语言和176B参数的大语言模型模型，其训练的架构如图所示，其使用了由48个英伟达 DGX-A100服务器组成的集群，GPU数=8*48共计384张，BLOOM训练采用的策略是首先将集群分为48个一组，进行数据并行。接下来，模型整体被分为12个阶段，进行流水线并行。每个阶段的模型被划分到4张GPU中进行张量并行。  

<img src="./images/BLOOM.png" style="zoom:70%;" /> 

NVIDIA DGX A100是一款极其强大的AI超级计算机。它装配了8块A100 GPU，能够提供高达5 PetaFLOPS的AI性能，从而可以处理最复杂和计算密集型的AI任务。DGX A100支持英伟达的多实例GPU（MIG）技术，可以将一个GPU分割为多个独立的GPU实例，这使得用户可以根据自己的需求，灵活地调整每个GPU实例的数量和大小，进一步提升硬件的利用率。此外，DGX A100还配备了全套的英伟达AI软件框架和库，如TensorFlow、PyTorch等，让用户可以立即开始AI的使用和开发，无需自己搭建软硬件环境  

<img src="./images/DGX-A100.png" style="zoom:30%;" />  

DGX A100参数  

<img src="./images/DGX-config.png" style="zoom:30%;" /> 

##### 计算设备内存优化

目前大模型训练通常采用Adam优化算法，其对计算设备内存的占用很大。为了降低内存占用，多数系统采用混合精度训练方式，即同时存在`FP16`或者`BF16`和`FP32`两种格式的数值。FP16的值区间比FP32要小很多，所以在计算过程中很容易出现上溢出和下溢出。BF16相比于FP16是以精度换更大的值区间范围，但是由于FP16和BF16的精度较低，训练过程中可能会出现梯度消失和模型不稳定的问题，因此需要动态损失缩放和混合精度优化器来解决这些问题。  

<img src="./images/FP16.png" style="zoom:60%;" /> 

混合精度优化过程也就是优化器及模型相关参数的精度是不一样的，并涉及到精度的转换。同时在训练过程中模型状态实际上需要的内存很大，可能是100G，因此也需要减少模型状态来解决内存占用问题。  

<img src="./images/混合精度.png" style="zoom:60%;" /> 

零冗余优化器(ZeRo)目标是针对模型状态的存储去除冗余的优化。ZeRO使用分区的方法，将模型状态量分割成多个分区，每个计算设备只保存其中的一部分。这个整个训练系统内只需要维护一份模型状态即可，减少了内存消耗和通信开销。  

<img src="./images/ZeRO.png" style="zoom:60%;" />



In [ ]:
# Pytorch中也实现了ZeRO优化方法

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from torch.distributed.optim import ZeroRedundancyOptimizer
from torch.nn.parallel import DistributedDataParallel as DDP

def print_peak_memory(prefix, device):
    